In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import openpyxl
import datetime
from sklearn import metrics
import re
from tqdm import tqdm_notebook as tqdm
import pandas_profiling


# Load the dataset
df = pd.read_excel('/Users/dawidarakelyan/Desktop/Capstone final/baza_final.xlsx', sheet_name='ապվճար')
df = pd.DataFrame(df)

# Rename columns
new_column_names = {'Ապահովագրության սկիզբ': 'Cont_start', 'Ապահովագրության ավարտ': 'Cont_end', 
                    'Ապահովագրության օրերի քանակ': 'Cont_days','Պատահարների գումար':'Target',
                    'մարզ':'Region','գործակալ':'Root'}
df = df.rename(columns=new_column_names)

# Generate a report of the dataset
pandas_profiling.ProfileReport(df)

# Convert columns to datetime format
df['bd'] = pd.to_datetime(df['bd'])
df['Cont_start'] = pd.to_datetime(df['Cont_start'])

# Calculate driver age at contract start
df['driver_age_at_contract_start'] = (df['Cont_start'] - df['bd']).astype('<m8[Y]')
df['d_age'] = df['driver_age_at_contract_start']

# Filter out drivers under 18 and over 90 years old
df = df[(df['d_age'] > 18) & (df['d_age'] < 90)]

# Calculate car age at contract start
df['contract_year'] = df['Cont_start'].dt.year
df['car_age_at_contract_start'] = (df['contract_year'] - df['car_year'])
df['car_year'] = df['car_age_at_contract_start']

# Filter out cars with negative or too large age
df = df[(df['car_year'] > 0) & (df['car_year'] < 200)]

# Drop unnecessary columns
columns_to_drop = ['bm', 'bd', 'summ', '1st acc date', 'last acc date', 'days from the first acc', 
                   'days from the first acc.1', 'տուժածների քանակ', 'պատահարների քանակ', 'հատուցում', 
                   'driver_age_at_contract_start', 'contract_year', 'car_age_at_contract_start']
df = df.drop(columns=columns_to_drop)

# Replace all 0's in 'Target' column with 1
df.loc[df['Target'] == 0, 'Target'] = 1
# Fill missing values in 'Target' column with 0
df['Target'] = df['Target'].fillna(0)
# Replace all non-zero values in 'Target' column with 1
df.loc[df["Target"] != 0, "Target"] = 1

# Keep only rows where 'type' column is 'CAR'
df = df[df['type'] == 'CAR']
# Remove 'type' column from the dataframe
df.drop("type", axis=1, inplace=True)

# Define a function to clean the car data
def clean_car_data(df):
    # Replace 'NIVA' in 'model' column with 'NIVA'
    df.loc[df['car'] == 'NIVA', 'model'] = 'NIVA'
    # Replace 'VAZ' in 'model' column with 'VAZ'
    df.loc[df['car'] == 'VAZ', 'model'] = 'VAZ'
    # Replace 'DAIMLER BENZ' in 'car' column with 'MERCEDES-BENZ'
    df.loc[df['car'] == 'DAIMLER BENZ', 'car'] = 'MERCEDES-BENZ'
    # Replace all cars in the list with 'RUS' in 'model' column
    df.loc[df['car'].isin(['LADA', 'DACIA', 'GAZ', 'UAZ', 'ZAZ', 'ERAZ', 'MOSKVICH', 'IZH']), 'model'] = 'RUS'
    return df

# Clean the car data using the 'clean_car_data' function
df = clean_car_data(df)

# Define a function to set the 'RUS' model for all Russian cars
def set_russian_model(df):
    # Replace all Russian cars in the list with 'RUS' in 'model' column
    df.loc[df['car'].isin(['LADA', 'DACIA', 'GAZ', 'UAZ', 'ZAZ', 'ERAZ', 'MOSKVICH', 'IZH']), 'model'] = 'RUS'
    return df

# Apply the 'set_russian_model' function on the dataframe
df = set_russian_model(df)

# Remove all rows where 'car' column is missing
df.dropna(subset=['car'], inplace=True)

# Convert 'car' column to string
df['car'] = df['car'].astype(str)

# Get the number of unique car brands in the dataframe
brands = df['car'].unique().tolist()
len_brands = len(brands)

# Loop through each row in the dataframe
for index, row in df.iterrows():
    # If the car brand is part of the model name
    if str(row['car']) in str(row['model']):
        # Remove the car brand from the model name
        new_model = row['model'].replace(str(row['car']), '').strip()
        # Update the model name in the dataframe
        df.at[index, 'model'] = new_model

df1 =df[['id','plate']].copy()

# Extract each character in the 'plate' column and create new columns for each character
df1[['one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight']] = \
    df1['plate'].astype(str).str.extract(
        r'(\d)?(\d)?([A-Z])?([A-Z])?(\d)?(\d)?(\d)?(\d)?', expand=True)

# Convert selected columns to numeric and drop rows with NaN values
cols = ['one', 'two', 'five', 'six', 'seven']
for col in cols:
    df1[col] = pd.to_numeric(df1[col], errors='coerce')

cols = ['one', 'two', 'three', 'four', 'five', 'six', 'seven'] # include all extracted columns

df1.dropna(subset=cols, inplace=True) # drop rows with NaN values in selected columns

# Drop the 'eight' column
df1.drop('eight', axis=1, inplace=True)

# Create a new column named 'group' with empty strings
df1['group'] = ''       




## 2

##aa

df1.loc[(
    df1['one'] == df1['two'])& 
    (df1['three'] == df1['four']) & 
    (df1['five'] == df1['seven']) & 
    (df1['one'] != df1['five'])& 
    (df1['six'] == 0),
    'group'] = '2' 

df1.loc[(
    df1['one'] == df1['two'])& 
    (df1['three'] == df1['four']) & 
    (df1['five'] == df1['six']) & 
    (df1['one'] != df1['seven'])& 
    (df1['six'] != 0) &
    (df1['seven'] != 0),
    'group'] = '2'

df1.loc[(
    df1['one'] == df1['two'])& 
    (df1['three'] == df1['four']) & 
    (df1['seven'] == df1['six']) & 
    (df1['five'] != df1['seven'])& 
    (df1['five'] != 0) &
    (df1['six'] != 0),
    'group'] = '2'

df1.loc[(
    df1['one'] == df1['two'])& 
    (df1['three'] == df1['four']) & 
    (df1['seven'] == df1['five']) & 
    (df1['five'] != df1['six'])& 
    (df1['five'] != 0) &
    (df1['six'] != 0),
    'group'] = '2'

df1.loc[(
    df1['one'] == df1['two'])& 
    (df1['three'] != df1['four']) & 
    (df1['five'] == df1['seven']) & 
    (df1['one'] != df1['five'])& 
    (df1['six'] == 0),
    'group'] = '2' 

df1.loc[(
    df1['one'] == df1['two'])& 
    (df1['three'] != df1['four']) & 
    (df1['five'] == df1['six']) & 
    (df1['one'] != df1['seven'])& 
    (df1['six'] != 0) &
    (df1['seven'] != 0),
    'group'] = '2'

df1.loc[(
    df1['one'] == df1['two'])& 
    (df1['three'] != df1['four']) & 
    (df1['seven'] == df1['six']) & 
    (df1['five'] != df1['seven'])& 
    (df1['five'] != 0) &
    (df1['six'] != 0),
    'group'] = '2'

df1.loc[(
    df1['one'] == df1['two'])& 
    (df1['three'] != df1['four']) & 
    (df1['seven'] == df1['five']) & 
    (df1['five'] != df1['six'])& 
    (df1['five'] != 0) &
    (df1['six'] != 0),
    'group'] = '2'

## 3


df1.loc[(
    df1['one'] ==0)& 
    ( df1['two']!=0)&
    (df1['three'] != df1['four']) & 
    (df1['five'] != df1['two']) & 
    (df1['five'] != 0) &
    (df1['six'] == df1['seven'])& 
    (df1['six'] == 0),
    'group'] = '3' 

df1.loc[
    (df1['one'] == df1['two'])& 
    (df1['three'] != df1['four']) & 
    (df1['seven'] == df1['six']) & 
    (df1['five'] != df1['six'])& 
    (df1['five'] !=df1['one'])&
    (df1['five'] != 0) &
    (df1['six'] == 0),
    'group'] = '3'

df1.loc[
    (df1['one'] == df1['two'])& 
    (df1['three'] != df1['four']) & 
    (df1['one'] != df1['seven'])& 
    (df1['five'] == df1['six']) & 
    (df1['five'] != df1['seven'])& 
    (df1['seven'] != 0) &
    (df1['six'] == 0),
    'group'] = '3'

df1.loc[
    (df1['one'] == df1['two'])& 
    (df1['three'] != df1['four']) & 
    (df1['one'] != df1['seven'])& 
    (df1['five'] != df1['six']) & 
    (df1['six'] == df1['seven'])& 
    (df1['seven'] != 0) &
    (df1['five'] == 0),
    'group'] = '3'

df1.loc[
    (df1['one'] != 0)& 
    (df1['two'] == 0)& 
    (df1['three'] != df1['four']) & 
    (df1['five'] != df1['six']) & 
    (df1['five'] == df1['seven'])& 
    (df1['seven'] == 0) &
    (df1['six'] != df1['one']),
    'group'] = '3'

df1.loc[
    (df1['one'] != 0)& 
    (df1['two'] == 0)& 
    (df1['three'] != df1['four']) & 
    (df1['five']== df1['six']) & 
    (df1['five'] != df1['seven'])& 
    (df1['seven'] != df1['one']) &
    (df1['six'] == 0),
    'group'] = '3'

df1.loc[
    (df1['one'] != 0)& 
    (df1['two'] == 0)& 
    (df1['three'] != df1['four']) & 
    (df1['seven']== df1['six']) & 
    (df1['five'] != df1['seven'])& 
    (df1['seven'] == 0) &
    (df1['five'] != df1['one']),
    'group'] = '3'

df1.loc[
    (df1['one'] != 0)& 
    (df1['two'] == 0)& 
    (df1['three'] != df1['four']) & 
    (df1['seven']== df1['six']) & 
    (df1['five'] == df1['seven'])& 
    (df1['seven'] != 0) &
    (df1['five'] != df1['one']),
    'group'] = '3'

## 4


df1.loc[(
    df1['one'] ==0)& 
    ( df1['two']!=0)&
    (df1['three'] == df1['four']) & 
    (df1['five'] != df1['two']) & 
    (df1['five'] != 0) &
    (df1['six'] == df1['seven'])& 
    (df1['six'] == 0),
    'group'] = '4' 

df1.loc[
    (df1['one'] == df1['two'])& 
    (df1['three'] == df1['four']) & 
    (df1['seven'] == df1['six']) & 
    (df1['five'] != df1['six'])& 
    (df1['five'] !=df1['one'])&
    (df1['five'] != 0) &
    (df1['six'] == 0),
    'group'] = '4'

df1.loc[
    (df1['one'] == df1['two'])& 
    (df1['three'] == df1['four']) & 
    (df1['one'] != df1['seven'])& 
    (df1['five'] == df1['six']) & 
    (df1['five'] != df1['seven'])& 
    (df1['seven'] != 0) &
    (df1['six'] == 0),
    'group'] = '4'

df1.loc[
    (df1['one'] == df1['two'])& 
    (df1['three'] == df1['four']) & 
    (df1['one'] != df1['seven'])& 
    (df1['five'] != df1['six']) & 
    (df1['six'] == df1['seven'])& 
    (df1['seven'] != 0) &
    (df1['five'] == 0),
    'group'] = '4'

df1.loc[
    (df1['one'] != 0)& 
    (df1['two'] == 0)& 
    (df1['three'] == df1['four']) & 
    (df1['five'] != df1['six']) & 
    (df1['five'] == df1['seven'])& 
    (df1['seven'] == 0) &
    (df1['six'] != df1['one']),
    'group'] = '4'

df1.loc[
    (df1['one'] != 0)& 
    (df1['two'] == 0)& 
    (df1['three'] == df1['four']) & 
    (df1['five']== df1['six']) & 
    (df1['five'] != df1['seven'])& 
    (df1['seven'] != df1['one']) &
    (df1['six'] == 0),
    'group'] = '4'

df1.loc[
    (df1['one'] != 0)& 
    (df1['two'] == 0)& 
    (df1['three'] == df1['four']) & 
    (df1['seven']== df1['six']) & 
    (df1['five'] != df1['seven'])& 
    (df1['seven'] == 0) &
    (df1['five'] != df1['one']),
    'group'] = '4'

df1.loc[
    (df1['one'] != 0)& 
    (df1['two'] == 0)& 
    (df1['three'] == df1['four']) & 
    (df1['seven']== df1['six']) & 
    (df1['five'] == df1['seven'])& 
    (df1['seven'] != 0) &
    (df1['five'] != df1['one']),
    'group'] = '4'

####

df1.loc[(
    df1['one'] !=0)& 
    ( df1['two']!=0)&
    ( df1['two']!=df1['one'])&
    (df1['three'] == df1['four']) & 
    (df1['five'] == df1['two']) & 
    (df1['five'] == df1['six']) &
    (df1['six'] == df1['seven']),
    'group'] = '4' 

df1.loc[(
    df1['one'] !=0)& 
    ( df1['two']!=0)&
    ( df1['two']!=df1['one'])&
    (df1['three'] != df1['four']) & 
    (df1['five'] == df1['two']) & 
    (df1['five'] == df1['six']) &
    (df1['six'] == df1['seven']),
    'group'] = '4' 

df1.loc[(
    df1['one'] !=0)& 
    ( df1['two']!=0)&
    ( df1['two']!=df1['one'])&
    (df1['three'] == df1['four']) & 
    (df1['five'] == df1['one']) & 
    (df1['five'] == df1['six']) &
    (df1['six'] == df1['seven']),
    'group'] = '4' 

df1.loc[(
    df1['one'] !=0)& 
    ( df1['two']!=0)&
    ( df1['two']!=df1['one'])&
    (df1['three'] != df1['four']) & 
    (df1['five'] == df1['one']) & 
    (df1['five'] == df1['six']) &
    (df1['six'] == df1['seven']),
    'group'] = '4' 

df1.loc[
    (df1['one'] ==0)& 
    ( df1['two']!=0)&
    (df1['three'] != df1['four']) & 
    (df1['five'] == 0) & 
    (df1['seven'] ==0) &
    (df1['six'] == df1['two']),
    'group'] = '4' 

df1.loc[
    (df1['one'] ==0)& 
    ( df1['two']!=0)&
    (df1['three'] != df1['four']) & 
    (df1['five'] == df1['two']) & 
    (df1['seven'] ==df1['five']) &
    (df1['six'] == 0),
    'group'] = '4' 

df1.loc[
    (df1['one'] ==0)& 
    ( df1['two']!=0)&
    (df1['three'] != df1['four']) & 
    (df1['five'] == 0) & 
    (df1['seven'] ==df1['two']) &
    (df1['six'] == df1['two']),
    'group'] = '4' 

df1.loc[
    (df1['one'] ==0)& 
    ( df1['two']!=0)&
    (df1['three'] != df1['four']) & 
    (df1['seven'] == 0) & 
    (df1['five'] ==df1['two']) &
    (df1['six'] == df1['two']),
    'group'] = '4' 

df1.loc[
    (df1['one'] !=0)& 
    ( df1['two']==df1['one'])&
    (df1['three'] != df1['four']) & 
    (df1['six'] == 0) & 
    (df1['five'] ==df1['seven']) &
    (df1['five'] == df1['two']),
    'group'] = '4' 

df1.loc[
    (df1['one'] !=0)& 
    ( df1['two']==df1['one'])&
    (df1['three'] != df1['four']) & 
    (df1['seven'] == 0) & 
    (df1['five'] ==df1['six']) &
    (df1['six'] == df1['two']),
    'group'] = '4' 

df1.loc[
    (df1['one'] !=0)& 
    ( df1['two']==0)&
    (df1['three'] != df1['four']) & 
    (df1['seven'] == 0) & 
    (df1['five'] == 0) & 
    (df1['one'] ==df1['six']) ,
    'group'] = '4' 

df1.loc[
    (df1['one'] !=0)& 
    ( df1['two']==0)&
    (df1['three'] != df1['four']) & 
    (df1['six'] == 0) & 
    (df1['five'] == df1['seven']) & 
    (df1['one'] ==df1['seven']) ,
    'group'] = '4' 

df1.loc[
    (df1['one'] !=0)& 
    ( df1['two']==0)&
    (df1['three'] != df1['four']) & 
    (df1['five'] == 0) & 
    (df1['six'] == df1['seven']) & 
    (df1['one'] ==df1['seven']) ,
    'group'] = '4' 

df1.loc[
    (df1['one'] !=0)& 
    ( df1['two']==0)&
    (df1['three'] != df1['four']) & 
    (df1['seven'] == 0) & 
    (df1['six'] == df1['five']) & 
    (df1['one'] ==df1['six']) ,
    'group'] = '4' 

df1.loc[
    (df1['one'] ==0)& 
    ( df1['two']!=0)&
    (df1['three'] != df1['four']) & 
    (df1['seven'] != 0) & 
    (df1['six'] == df1['five']) & 
    (df1['six'] == df1['seven']) & 
    (df1['two'] !=df1['six']) ,
    'group'] = '4' 

df1.loc[
    (df1['one'] ==0)& 
    ( df1['two']!=0)&
    (df1['three'] != df1['four']) & 
    (df1['seven'] != 0) & 
    (df1['six'] ==0) & 
    (df1['five'] ==0) & 
    (df1['two'] !=df1['seven']) ,
    'group'] = '4' 

df1.loc[
    (df1['one'] !=0)& 
    ( df1['two']==df1['one'])&
    (df1['three'] != df1['four']) & 
    (df1['seven'] == 0) & 
    (df1['five'] ==0) &  
    (df1['six'] !=df1['two']) ,
    'group'] = '4' 

## 5


df1.loc[
    (df1['one'] ==0)& 
    ( df1['two']!=0)&
    (df1['three'] == df1['four']) & 
    (df1['five'] ==0)&
    (df1['seven'] ==0)& 
    (df1['six'] == df1['two']),
    'group'] = '5' 

df1.loc[
    (df1['one'] ==0)& 
    ( df1['two']!=0)&
    (df1['three'] == df1['four']) & 
    (df1['six'] ==0)&
    (df1['seven'] ==0)& 
    (df1['five'] == df1['two']),
    'group'] = '5' 

df1.loc[
    (df1['one'] ==0)& 
    ( df1['two']!=0)&
    (df1['three'] == df1['four']) & 
    (df1['five'] ==0)& 
    (df1['two'] == df1['six'])&
    (df1['seven'] == df1['six']),
    'group'] = '5' 

df1.loc[
    (df1['one'] ==0)& 
    ( df1['two']!=0)&
    (df1['three'] == df1['four']) & 
    (df1['seven'] ==0)& 
    (df1['two'] == df1['six'])&
    (df1['five'] == df1['six']),
    'group'] = '5' 

df1.loc[
    (df1['one'] ==df1['two'])& 
    ( df1['two']!=0)&
    (df1['three'] == df1['four']) & 
    (df1['six'] ==0)& 
    (df1['two'] == df1['five'])&
    (df1['five'] == df1['seven']),
    'group'] = '5' 

df1.loc[
    (df1['one'] ==df1['two'])& 
    ( df1['two']!=0)&
    (df1['three'] == df1['four']) & 
    (df1['seven'] ==0)& 
    (df1['two'] == df1['five'])&
    (df1['five'] == df1['six']),
    'group'] = '5' 

df1.loc[
    (df1['two'] ==0)& 
    ( df1['one']!=0)&
    (df1['three'] == df1['four']) & 
    (df1['seven'] ==0)& 
    (df1['five'] ==0)& 
    (df1['one'] == df1['six']),
    'group'] = '5' 

df1.loc[
    (df1['two'] ==0)& 
    ( df1['one']!=0)&
    (df1['three'] == df1['four']) & 
    (df1['six'] ==0)& 
    (df1['five'] == df1['seven'])&
    (df1['one'] == df1['seven']),
    'group'] = '5' 

df1.loc[
    (df1['two'] ==0)& 
    ( df1['one']!=0)&
    (df1['three'] == df1['four']) & 
    (df1['five'] ==0)& 
    (df1['six'] == df1['seven'])&
    (df1['one'] == df1['seven']),
    'group'] = '5' 

df1.loc[
    (df1['two'] ==0)& 
    ( df1['one']!=0)&
    (df1['three'] == df1['four']) & 
    (df1['seven'] ==0)& 
    (df1['six'] == df1['five'])&
    (df1['one'] == df1['five']),
    'group'] = '5' 

df1.loc[
    ( df1['one']==0)&
    (df1['two'] !=0)& 
    (df1['three'] == df1['four']) & 
    (df1['six'] == df1['five'])&
    (df1['two'] != df1['five'])&
    (df1['six'] == df1['seven']),
    'group'] = '5' 

df1.loc[
    ( df1['one']==0)&
    (df1['two'] !=0)& 
    (df1['three'] == df1['four']) & 
    (df1['five'] ==0)& 
    (df1['six'] ==0)& 
    (df1['seven'] != df1['two'])&
    ( df1['seven']!=0),
    'group'] = '5' 

df1.loc[
    ( df1['one']!=0)&
    (df1['two'] ==df1['one'])& 
    (df1['three'] == df1['four']) & 
    (df1['five'] ==0)& 
    (df1['seven'] ==0)& 
    (df1['six'] != df1['two']),
    'group'] = '5' 

df1.loc[
    ( df1['one']!=0)&
    (df1['two'] ==df1['one'])& 
    (df1['three'] != df1['four']) & 
    (df1['five'] ==0)& 
    (df1['seven'] ==df1['two'])& 
    (df1['six'] == df1['two']),
    'group'] = '5' 

df1.loc[
    ( df1['one']!=0)&
    (df1['two'] ==df1['one'])& 
    (df1['three'] != df1['four']) & 
    (df1['five'] !=0)& 
    (df1['seven'] !=df1['two'])& 
    (df1['six'] == df1['five'])&
    (df1['six'] == df1['seven']),
    'group'] = '5' 

df1.loc[
    ( df1['one']!=0)&
    (df1['two'] ==df1['one'])& 
    (df1['three'] != df1['four']) & 
    (df1['five'] ==df1['one'])& 
    (df1['seven'] ==0)& 
    (df1['six'] ==0),
    'group'] = '5' 

## 6

df1.loc[
    ( df1['one']!=0)&
    (df1['two'] ==df1['one'])& 
    (df1['three'] == df1['four']) & 
    (df1['five'] ==0)&
    (df1['six'] ==df1['one'])& 
     (df1['seven'] ==df1['six']),
    'group'] = '6' 

df1.loc[
    ( df1['one']!=0)&
    (df1['two'] ==df1['one'])& 
    (df1['three'] == df1['four']) & 
    (df1['six'] !=df1['one'])& 
    (df1['seven'] ==df1['six'])&
     (df1['five'] ==df1['six']),
    'group'] = '6' 

df1.loc[
    ( df1['one']!=0)&
    (df1['two'] ==df1['one'])& 
    (df1['three'] == df1['four']) & 
    (df1['five'] ==df1['one'])& 
    (df1['seven'] ==0)&
     (df1['six'] ==0),
    'group'] = '6' 

## 7

df1.loc[
    ( df1['one']==0)&
    (df1['two'] !=0)& 
    (df1['three'] != df1['four']) & 
    (df1['five'] ==df1['two'])&
    (df1['six'] ==df1['two'])& 
     (df1['seven'] ==df1['two']),
    'group'] = '7' 

df1.loc[
    ( df1['one']==0)&
    (df1['two'] !=0)& 
    (df1['three'] != df1['four']) & 
    (df1['five'] ==df1['two'])&
    (df1['six'] ==0)& 
     (df1['seven'] ==0),
    'group'] = '7' 

df1.loc[
    ( df1['one']!=0)&
    (df1['two'] ==df1['one'])& 
    (df1['three'] != df1['four']) & 
    (df1['five'] ==0)&
    (df1['six'] ==0)& 
     (df1['seven'] ==df1['one']),
    'group'] = '7' 

df1.loc[
    ( df1['one']!=0)&
    (df1['two'] ==0)& 
    (df1['three'] != df1['four']) & 
    (df1['five'] ==df1['one'])&
    (df1['six'] ==df1['one'])& 
     (df1['seven'] ==df1['one']),
    'group'] = '7' 

df1.loc[
    ( df1['one']!=0)&
    (df1['two'] ==0)& 
    (df1['three'] != df1['four']) & 
    (df1['five'] ==df1['one'])&
    (df1['six'] ==0)& 
     (df1['seven'] ==0),
    'group'] = '7' 

##  8

df1.loc[
    ( df1['one']==0)&
    (df1['two'] !=0)& 
    (df1['three'] == df1['four']) & 
    (df1['five'] ==df1['two'])&
    (df1['six'] ==df1['two'])& 
     (df1['seven'] ==df1['two']),
    'group'] = '8' 

df1.loc[
    ( df1['one']==0)&
    (df1['two'] !=0)& 
    (df1['three'] == df1['four']) & 
    (df1['five'] ==df1['two'])&
    (df1['six'] ==0)& 
     (df1['seven'] ==0),
    'group'] = '8' 

df1.loc[
    ( df1['one']!=0)&
    (df1['two'] ==df1['one'])& 
    (df1['three'] == df1['four']) & 
    (df1['five'] ==0)&
    (df1['six'] ==0)& 
     (df1['seven'] ==df1['one']),
    'group'] = '8' 

df1.loc[
    ( df1['one']!=0)&
    (df1['two'] ==0)& 
    (df1['three'] == df1['four']) & 
    (df1['five'] ==df1['one'])&
    (df1['six'] ==df1['one'])& 
     (df1['seven'] ==df1['one']),
    'group'] = '8' 

df1.loc[
    ( df1['one']!=0)&
    (df1['two'] ==0)& 
    (df1['three'] == df1['four']) & 
    (df1['five'] ==df1['one'])&
    (df1['six'] ==0)& 
     (df1['seven'] ==0),
    'group'] = '8' 

##  9

df1.loc[
    ( df1['one']==0)&
    (df1['two'] !=0)& 
    (df1['three'] != df1['four']) & 
    (df1['five'] ==0)&
    (df1['six'] ==0)& 
     (df1['seven'] ==df1['two']),
    'group'] = '9' 


df1.loc[
    ( df1['one']!=0)&
    (df1['two'] ==0)& 
    (df1['three'] != df1['four']) & 
    (df1['five'] ==0)&
    (df1['six'] ==0)&
     (df1['seven'] ==df1['one']),
    'group'] = '9' 

##  10

df1.loc[
    ( df1['one']!=0)&
    (df1['two'] ==df1['one'])& 
    (df1['three'] != df1['four']) & 
    (df1['five'] ==df1['one'])&
    (df1['six'] ==df1['one'])& 
     (df1['seven'] ==df1['one']),
    'group'] = '10' 

##  11

df1.loc[
    ( df1['one']==0)&
    (df1['two'] !=0)& 
    (df1['three'] == df1['four']) & 
    (df1['five'] ==0)&
    (df1['six'] ==0)& 
     (df1['seven'] ==df1['two']),
    'group'] = '11' 

df1.loc[
    ( df1['one']!=0)&
    (df1['two'] ==0)& 
    (df1['three'] == df1['four']) & 
    (df1['five'] ==0)&
    (df1['six'] ==0)& 
     (df1['seven'] ==df1['one']),
    'group'] = '11' 

##  12

df1.loc[
    ( df1['one']!=0)&
    (df1['two'] ==df1['one'])& 
    (df1['three'] == df1['four']) & 
    (df1['five'] ==df1['one'])&
    (df1['six'] ==df1['one'])& 
     (df1['seven'] ==df1['one']),
    'group'] = '12' 

# Replace empty strings with NaN
df1 = df1.replace(r'^\s*$', np.nan, regex=True)

# Copy required columns into a new dataframe
df2 = df1[['id','plate','group']].copy()

# Fill NaN values in the 'group' column with 1
df2['group'] = df2['group'].fillna(1)

# Merge the two dataframes on 'id' column using a left join
res = df2.merge(df, on='id', how='left')

# Replace empty strings with NaN values
df1 = df1.replace(r'^\s*$', np.nan, regex=True)

# Create a copy of columns 'id', 'plate', and 'group' from df1 into df2
df2 = df1[['id','plate','group']].copy()

# Fill NaN values in 'group' with 1
df2['group'] = df2['group'].fillna(1)

# Merge df2 with df on 'id' column with a left join
res = df2.merge(df, on='id', how='left')

# Count the number of occurrences of each 'car' value
car_count = res["car"].value_counts()

# Select 'car' values with a count of less than 83
less_than_83 = car_count[car_count < 84].index

# Filter the res DataFrame to include only rows with 'car' values not in less_than_83
df_filtered = res[~res['car'].isin(less_than_83)]

# Group by 'car' and 'hp' columns, and forward fill 'model' column within each group
df_filtered['model'] = df_filtered.groupby(['car', 'hp'])['model'].fillna(method='ffill')

# Drop any remaining rows with NaN values in the 'model' column
df_filtered = df_filtered.dropna(subset=['model'])

bmw=df_filtered.copy()

import re

# A dictionary of BMW models and their corresponding model numbers
BMW_MODELS = {
    "C": ["116", "118", "120", "135", "218", "228"],
    "D": [
        "316", "318", "320", "323", "324", "325", "328", "330", "335", "335I", "316I",
        "318TI", "335XI", "330CI", "M3"
    ],
    "E": [
        "518", "520", "523", "524", "525", "528", "530", "535", "540", "545", "550",
        "550I", "M5", "628", "630", "633", "635", "640", "640I", "645", "650"
    ],
    "F": [
        "7", "728", "730", "735", "740", "745", "750", "750LI", "760", "725TDS", "M7"
    ],
    "S": ["840", "850", "Z3", "Z4", "Z8", "M8"],
    "S SUV": ["X1", "X3", "X4"],
    "M SUV": ["X5", "X6", "X6M"],
    "L SUV": ["X7"]
}

def classify_bmw_models(row):
    """
    Classifies the given BMW model into a category based on its model number.

    Args:
        row: A pandas DataFrame row object containing the 'model' column.

    Returns:
        A string representing the category of the BMW model.
    """
    model = str(row['model']).strip()

    if model:
        # Find the model number in the model string
        model_num = re.findall(r'(?<![A-Za-z])[Xx]?\d+[A-Za-z]?(?![A-Za-z])', model)

        if model_num and model_num[0] in [num for nums in BMW_MODELS.values() for num in nums]:
            # If the model number is found in the BMW_MODELS dictionary, classify the model
            for model_class, model_nums in BMW_MODELS.items():
                if model_num[0] in model_nums:
                    return model_class
        else:
            # If the model number is not found in the BMW_MODELS dictionary, classify as 'Other'
            return 'Other'
    else:
        # If the 'model' column is empty, classify as 'Other'
        return 'Other'

# Classify all BMW models in the 'car' column and add the classification to a new 'car_class' column
bmw.loc[bmw['car'] == 'BMW', 'car_class'] = bmw.loc[bmw['car'] == 'BMW'].apply(classify_bmw_models, axis=1)

# Classify specific BMW models based on their model numbers and update the 'car_class' column
bmw.loc[(bmw['car'] == 'BMW') & (bmw['model'].str.contains('318TI', case=False)), 'car_class'] = 'D'
bmw.loc[(bmw['car'] == 'BMW') & (bmw['model'].str.contains('335XI', case=False)), 'car_class'] = 'D'
bmw.loc[(bmw['car'] == 'BMW') & (bmw['model'].str.contains('528XI', case=False)), 'car_class'] = 'E'
bmw.loc[(bmw['car'] == 'BMW') & (bmw['model'].str.contains('Z', case=False)), 'car_class'] = 'S'
bmw.loc[(bmw['car'] == 'BMW') & (bmw['model'].str.contains('Z3', case=False)), 'car_class'] = 'S'
bmw.loc[(bmw['car'] == 'BMW') & (bmw['model'].str.contains('Z4', case=False)), 'car_class'] = 'S'
bmw.loc[(bmw['car'] == 'BMW') & (bmw['model'].str.contains('Z 4', case=False)), 'car_class'] = 'S'
bmw.loc[(bmw['car'] == 'BMW') & (bmw['model'].str.contains('725TDS', case=False)), 'car_class'] = 'F'
bmw.loc[(bmw['car'] == 'BMW') & (bmw['model'].str.contains('725', case=False)), 'car_class'] = 'F'
bmw.loc[(bmw['car'] == 'BMW') & (bmw['model'].str.contains('M3', case=False)), 'car_class'] = 'D'
bmw.loc[(bmw['car'] == 'BMW') & (bmw['model'].str.contains('330CI', case=False)), 'car_class'] = 'D'
bmw.loc[(bmw['car'] == 'BMW') & (bmw['model'].str.contains('M5', case=False)), 'car_class'] = 'E'
bmw.loc[(bmw['car'] == 'BMW') & (bmw['model'].str.contains('750LI', case=False)), 'car_class'] = 'F'


# Dictionary of Opel car models and their classes
opel_models = {
    "A": ["AGILA"],
    "B": ["CORSA", "VITA", "MERIVA"],
    "C": ["ASTRA", "ZAFIRA", "SINTRA"],
    "D": ["VECTRA", "REKORD", "ASCONA", "SIGNUM"],
    "E": ["OMEGA", "SENATOR"],
    "S": ["CALIBRA"],
    "VAN": ["COMBO", "VIVARO"],
    "M SUV": ["ANTARA", "FRONTERA", "MONTEREY"]
}

def classify_opel_models(row):
    """
    This function classifies Opel car models into their respective classes
    based on the opel_models dictionary.
    
    Parameters:
    row (pandas.core.series.Series): A row of the bmw DataFrame.
    
    Returns:
    str: The Opel car class of the model in the given row.
    """
    model = str(row["model"]).strip()
    if model:
        # Find all the alphanumeric sequences in the model string
        model_num = re.findall(r"[^\W\d_]+|\d+", model)
        # Check if any of the model names from the dictionary is in the alphanumeric sequence
        for model_class, model_names in opel_models.items():
            for name in model_names:
                if name in model_num:
                    return model_class
        else:
            # If the model does not match any of the model names from the dictionary, classify it as "Other"
            return "Other"
    else:
        # If the model string is empty, classify it as "Other"
        return "Other"

# Classify Opel car models into their respective classes using the classify_opel_models function
bmw.loc[bmw["car"] == "OPEL", "car_class"] = bmw.loc[bmw["car"] == "OPEL"].apply(classify_opel_models, axis=1)

# Classify specific Opel car models into their respective classes based on their names
bmw.loc[(bmw["car"] == "OPEL") & (bmw["model"].str.contains("ADMIRAL", case=False)), "car_class"] = "F"
bmw.loc[(bmw["car"] == "OPEL") & (bmw["model"].str.contains("ASCONA", case=False)), "car_class"] = "D"
bmw.loc[(bmw["car"] == "OPEL") & (bmw["model"].str.contains("TIGRA", case=False)), "car_class"] = "C"
bmw.loc[(bmw["car"] == "OPEL") & (bmw["model"].str.contains("REKORD", case=False)), "car_class"] = "D"
bmw.loc[(bmw["car"] == "OPEL") & (bmw["model"].str.contains("KADETT", case=False)), "car_class"] = "C"
bmw.loc[(bmw["car"] == "OPEL") & (bmw["model"].str.contains("RECORD", case=False)), "car_class"] = "D"
bmw.loc[(bmw["car"] == "OPEL") & (bmw["model"].str.contains("SIGNUM", case=False)), "car_class"] = "D"
bmw.loc[(bmw['car'] == 'OPEL') & (bmw['model'].str.contains("SENATOR", case=False)), 'car_class'] = 'E'


# Define a mask for filtering Mercedes-Benz cars
mercedes_mask = df_filtered['car'] == 'MERCEDES-BENZ'

# Remove certain words from the model names of Mercedes-Benz cars
bmw.loc[mercedes_mask, 'model'] = (
    bmw.loc[mercedes_mask, 'model']
    .str.replace('AMG', '')
    .str.replace('4MATIC', '')
    .str.replace('KOMPRESSOR', '')
    .str.replace('COUPE', '')
    .str.replace('4 MATIC', '')
    .str.replace('TD', '')
    .str.replace('CDI', '')
    .str.replace('CGI', '')
)

# Define a dictionary that maps Mercedes-Benz models to their class
mercedes_models = {
    "A": ["SMART"],
    "B": ['B'],
    "C": ["D", "A"],
    "D": ["C"],
    "E": ["E", 'W123', 'CLS'],
    "F": ["S",],
    "S": ["SLK", 'SL', 'CL'],
    "M": ["VITO-V-280", 'VITO', 'VIANO', 'VANEO', 'V', 'SPRINTER', 'R'],
    "S SUV": ["GLC", "GLK"],
    "M SUV": ["ML", "GLE", "G"],
    "L SUV": ["GLS"]
}

# Define a function to classify Mercedes-Benz models based on the dictionary
def classify_mercedes_models(row):
    model = str(row['model']).strip()
    if model:
        for model_class, model_names in mercedes_models.items():
            if any(name in model.upper() for name in model_names):
                return model_class
        else:
            return 'Other'
    else:
        return 'Other'

# Apply the function to classify Mercedes-Benz models
bmw.loc[bmw['car'] == 'MERCEDES-BENZ', 'car_class'] = (
    bmw.loc[bmw['car'] == 'MERCEDES-BENZ']
    .apply(classify_mercedes_models, axis=1)
)

# Manually classify certain models of Mercedes-Benz cars
bmw.loc[(bmw['car'] == 'MERCEDES-BENZ') & (bmw['model'].str.contains('SPRINTER', case=False)), 'car_class'] = 'M'
bmw.loc[(bmw['car'] == 'MERCEDES-BENZ') & (bmw['model'].str.contains('VITO', case=False)), 'car_class'] = 'M'
bmw.loc[(bmw['car'] == 'MERCEDES-BENZ') & (bmw['model'].str.contains('VIANO', case=False)), 'car_class'] = 'M'
bmw.loc[(bmw['car'] == 'MERCEDES-BENZ') & (bmw['model'].str.contains('VANEO', case=False)), 'car_class'] = 'M'

bmw.loc[(bmw['car'] == 'NISSAN') & (bmw['model'].str.contains('TIIDA', case=False)), 'car_class'] = 'C'
bmw.loc[(bmw['car'] == 'NISSAN') & (bmw['model'].str.contains('TEANA', case=False)), 'car_class'] = 'D'
bmw.loc[(bmw['car'] == 'NISSAN') & (bmw['model'].str.contains('MARCH', case=False)), 'car_class'] = 'B'
bmw.loc[(bmw['car'] == 'NISSAN') & (bmw['model'].str.contains('X-TRAIL', case=False)), 'car_class'] = 'S SUV'
bmw.loc[(bmw['car'] == 'NISSAN') & (bmw['model'].str.contains('X TRAIL', case=False)), 'car_class'] = 'S SUV'
bmw.loc[(bmw['car'] == 'NISSAN') & (bmw['model'].str.contains('X TERRA', case=False)), 'car_class'] = 'L SUV'
bmw.loc[(bmw['car'] == 'NISSAN') & (bmw['model'].str.contains('VERSA', case=False)), 'car_class'] = 'C'
bmw.loc[(bmw['car'] == 'NISSAN') & (bmw['model'].str.contains('BLUEBIRD', case=False)), 'car_class'] = 'C'
bmw.loc[(bmw['car'] == 'NISSAN') & (bmw['model'].str.contains('NOTE', case=False)), 'car_class'] = 'B'
bmw.loc[(bmw['car'] == 'NISSAN') & (bmw['model'].str.contains('SKYLINE', case=False)), 'car_class'] = 'D'
bmw.loc[(bmw['car'] == 'NISSAN') & (bmw['model'].str.contains('ELGRAND', case=False)), 'car_class'] = 'MPV'
bmw.loc[(bmw['car'] == 'NISSAN') & (bmw['model'].str.contains('ALTIMA', case=False)), 'car_class'] = 'D'
bmw.loc[(bmw['car'] == 'NISSAN') & (bmw['model'].str.contains('X-TERRA', case=False)), 'car_class'] = 'L SUV'
bmw.loc[(bmw['car'] == 'NISSAN') & (bmw['model'].str.contains('MURANO', case=False)), 'car_class'] = 'M SUV'
bmw.loc[(bmw['car'] == 'NISSAN') & (bmw['model'].str.contains('PATHFINDER', case=False)), 'car_class'] = 'L SUV'
bmw.loc[(bmw['car'] == 'NISSAN') & (bmw['model'].str.contains('FUGA', case=False)), 'car_class'] = 'E'
bmw.loc[(bmw['car'] == 'NISSAN') & (bmw['model'].str.contains('ROGUE', case=False)), 'car_class'] = 'S SUV'
bmw.loc[(bmw['car'] == 'NISSAN') & (bmw['model'].str.contains('SUNNY', case=False)), 'car_class'] = 'C'
bmw.loc[(bmw['car'] == 'NISSAN') & (bmw['model'].str.contains('350Z', case=False)), 'car_class'] = 'S'
bmw.loc[(bmw['car'] == 'NISSAN') & (bmw['model'].str.contains('JUKE', case=False)), 'car_class'] = 'B'
bmw.loc[(bmw['car'] == 'NISSAN') & (bmw['model'].str.contains('ALMERA ', case=False)), 'car_class'] = 'C'
bmw.loc[(bmw['car'] == 'NISSAN') & (bmw['model'].str.contains('CUBE ', case=False)), 'car_class'] = 'B'
bmw.loc[(bmw['car'] == 'NISSAN') & (bmw['model'].str.contains('SENTRA', case=False)), 'car_class'] = 'C'
bmw.loc[(bmw['car'] == 'NISSAN') & (bmw['model'].str.contains('ARMADA', case=False)), 'car_class'] = 'L SUV'
bmw.loc[(bmw['car'] == 'NISSAN') & (bmw['model'].str.contains('MICRA', case=False)), 'car_class'] = 'B'
bmw.loc[(bmw['car'] == 'NISSAN') & (bmw['model'].str.contains('QASHQAI ', case=False)), 'car_class'] = 'C'
bmw.loc[(bmw['car'] == 'NISSAN') & (bmw['model'].str.contains('PRIMERA ', case=False)), 'car_class'] = 'D'
bmw.loc[(bmw['car'] == 'NISSAN') & (bmw['model'].str.contains('MAXIMA', case=False)), 'car_class'] = 'D'
bmw.loc[(bmw['car'] == 'NISSAN') & (bmw['model'].str.contains('TERRANO', case=False)), 'car_class'] = 'M SUV'
bmw.loc[(bmw['car'] == 'NISSAN') & (bmw['model'].str.contains('SERENA', case=False)), 'car_class'] = 'MPV'
bmw.loc[(bmw['car'] == 'NISSAN') & (bmw['model'].str.contains('QUEST', case=False)), 'car_class'] = 'MPV'
bmw.loc[(bmw['car'] == 'NISSAN') & (bmw['model'].str.contains('PRESAGE', case=False)), 'car_class'] = 'MPV'
bmw.loc[(bmw['car'] == 'NISSAN') & (bmw['model'].str.contains('PATROL', case=False)), 'car_class'] = 'L SUV'

bmw.loc[(bmw['car'] == 'TOYOTA') & (bmw['model'].str.contains('CAMRY', case=False)), 'car_class'] = 'D'
bmw.loc[(bmw['car'] == 'TOYOTA') & (bmw['model'].str.contains('COROLLA', case=False)), 'car_class'] = 'C'
bmw.loc[(bmw['car'] == 'TOYOTA') & (bmw['model'].str.contains('RAV', case=False)), 'car_class'] = 'S SUV'
bmw.loc[(bmw['car'] == 'TOYOTA') & (bmw['model'].str.contains('ALPHARD ', case=False)), 'car_class'] = 'MPV'
bmw.loc[(bmw['car'] == 'TOYOTA') & (bmw['model'].str.contains('IPSUM', case=False)), 'car_class'] = 'MPV'
bmw.loc[(bmw['car'] == 'TOYOTA') & (bmw['model'].str.contains('VITZ', case=False)), 'car_class'] = 'B'
bmw.loc[(bmw['car'] == 'TOYOTA') & (bmw['model'].str.contains('ALTEZZA', case=False)), 'car_class'] = 'D'
bmw.loc[(bmw['car'] == 'TOYOTA') & (bmw['model'].str.contains('LAND CRUISER', case=False)), 'car_class'] = 'L SUV'
bmw.loc[(bmw['car'] == 'TOYOTA') & (bmw['model'].str.contains('LC', case=False)), 'car_class'] = 'L SUV'
bmw.loc[(bmw['car'] == 'TOYOTA') & (bmw['model'].str.contains('HARRIER', case=False)), 'car_class'] = 'M SUV'
bmw.loc[(bmw['car'] == 'TOYOTA') & (bmw['model'].str.contains('VOXY', case=False)), 'car_class'] = 'MPV'
bmw.loc[(bmw['car'] == 'TOYOTA') & (bmw['model'].str.contains('BELTA', case=False)), 'car_class'] = 'B'
bmw.loc[(bmw['car'] == 'TOYOTA') & (bmw['model'].str.contains('SIENNA', case=False)), 'car_class'] = 'MPV'
bmw.loc[(bmw['car'] == 'TOYOTA') & (bmw['model'].str.contains('SIENTA', case=False)), 'car_class'] = 'MPV'
bmw.loc[(bmw['car'] == 'TOYOTA') & (bmw['model'].str.contains('PRIUS', case=False)), 'car_class'] = 'D'
bmw.loc[(bmw['car'] == 'TOYOTA') & (bmw['model'].str.contains('PLATZ', case=False)), 'car_class'] = 'B'
bmw.loc[(bmw['car'] == 'TOYOTA') & (bmw['model'].str.contains('NOAH', case=False)), 'car_class'] = 'MPV'
bmw.loc[(bmw['car'] == 'TOYOTA') & (bmw['model'].str.contains('4RUNNER', case=False)), 'car_class'] = 'M SUV'
bmw.loc[(bmw['car'] == 'TOYOTA') & (bmw['model'].str.contains('4 RUNNER', case=False)), 'car_class'] = 'M SUV'
bmw.loc[(bmw['car'] == 'TOYOTA') & (bmw['model'].str.contains('FJ', case=False)), 'car_class'] = 'M SUV'
bmw.loc[(bmw['car'] == 'TOYOTA') & (bmw['model'].str.contains('HIGHLANDER', case=False)), 'car_class'] = 'M SUV'
bmw.loc[(bmw['car'] == 'TOYOTA') & (bmw['model'].str.contains('SEQUOIA', case=False)), 'car_class'] = 'L SUV'
bmw.loc[(bmw['car'] == 'TOYOTA') & (bmw['model'].str.contains('FORTUNER', case=False)), 'car_class'] = 'M SUV'
bmw.loc[(bmw['car'] == 'TOYOTA') & (bmw['model'].str.contains('ISIS', case=False)), 'car_class'] = 'MPV'
bmw.loc[(bmw['car'] == 'TOYOTA') & (bmw['model'].str.contains('PRADO', case=False)), 'car_class'] = 'L SUV'
bmw.loc[(bmw['car'] == 'TOYOTA') & (bmw['model'].str.contains('CROWN', case=False)), 'car_class'] = 'E' 
bmw.loc[(bmw['car'] == 'TOYOTA') & (bmw['model'].str.contains('AVALON', case=False)), 'car_class'] = 'E'
bmw.loc[(bmw['car'] == 'TOYOTA') & (bmw['model'].str.contains('AVENSIS', case=False)), 'car_class'] = 'D'
bmw.loc[(bmw['car'] == 'TOYOTA') & (bmw['model'].str.contains('HILUX', case=False)), 'car_class'] = 'PICKUP'
bmw.loc[(bmw['car'] == 'TOYOTA') & (bmw['model'].str.contains('YARIS', case=False)), 'car_class'] = 'B'
bmw.loc[(bmw['car'] == 'TOYOTA') & (bmw['model'].str.contains('MARK', case=False)), 'car_class'] = 'D'
bmw.loc[(bmw['car'] == 'TOYOTA') & (bmw['model'].str.contains('TERCEL', case=False)), 'car_class'] = 'B'
bmw.loc[(bmw['car'] == 'TOYOTA') & (bmw['model'].str.contains('SCION', case=False)), 'car_class'] = 'B'
bmw.loc[(bmw['car'] == 'TOYOTA') & (bmw['model'].str.contains('TUNDRA', case=False)), 'car_class'] = 'PICKUP'
bmw.loc[(bmw['car'] == 'TOYOTA') & (bmw['model'].str.contains('PROBOX', case=False)), 'car_class'] = 'M'

bmw.loc[(bmw['car'] == 'VOLKSWAGEN') & (bmw['model'].str.contains('PASSAT', case=False)), 'car_class'] = 'D'
bmw.loc[(bmw['car'] == 'VOLKSWAGEN') & (bmw['model'].str.contains('GOLF', case=False)), 'car_class'] = 'C'
bmw.loc[(bmw['car'] == 'VOLKSWAGEN') & (bmw['model'].str.contains('VENTO', case=False)), 'car_class'] = 'C'
bmw.loc[(bmw['car'] == 'VOLKSWAGEN') & (bmw['model'].str.contains('JETTA', case=False)), 'car_class'] = 'C'
bmw.loc[(bmw['car'] == 'VOLKSWAGEN') & (bmw['model'].str.contains('BORA', case=False)), 'car_class'] = 'C'
bmw.loc[(bmw['car'] == 'VOLKSWAGEN') & (bmw['model'].str.contains('TOUAREG', case=False)), 'car_class'] = 'M SUV'
bmw.loc[(bmw['car'] == 'VOLKSWAGEN') & (bmw['model'].str.contains('POLO', case=False)), 'car_class'] = 'B'
bmw.loc[(bmw['car'] == 'VOLKSWAGEN') & (bmw['model'].str.contains('TIGUAN', case=False)), 'car_class'] = 'S SUV'
bmw.loc[(bmw['car'] == 'VOLKSWAGEN') & (bmw['model'].str.contains('TRANSPORTER', case=False)), 'car_class'] = 'M'
bmw.loc[(bmw['car'] == 'VOLKSWAGEN') & (bmw['model'].str.contains('SCIROCCO', case=False)), 'car_class'] = 'C'
bmw.loc[(bmw['car'] == 'VOLKSWAGEN') & (bmw['model'].str.contains('BEETLE', case=False)), 'car_class'] = 'C'
bmw.loc[(bmw['car'] == 'VOLKSWAGEN') & (bmw['model'].str.contains('CARAVELLE', case=False)), 'car_class'] = 'MPV'

bmw.loc[(bmw['car'] == 'MITSUBISHI') & (bmw['model'].str.contains('PAJERO', case=False)), 'car_class'] = 'L SUV'
bmw.loc[(bmw['car'] == 'MITSUBISHI') & (bmw['model'].str.contains('MONTERO', case=False)), 'car_class'] = 'L SUV'
bmw.loc[(bmw['car'] == 'MITSUBISHI') & (bmw['model'].str.contains('GRANDIS', case=False)), 'car_class'] = 'MPV'
bmw.loc[(bmw['car'] == 'MITSUBISHI') & (bmw['model'].str.contains('LANCER', case=False)), 'car_class'] = 'C'
bmw.loc[(bmw['car'] == 'MITSUBISHI') & (bmw['model'].str.contains('OUTLANDER', case=False)), 'car_class'] = 'S SUV'
bmw.loc[(bmw['car'] == 'MITSUBISHI') & (bmw['model'].str.contains('AIRTREK', case=False)), 'car_class'] = 'S SUV'
bmw.loc[(bmw['car'] == 'MITSUBISHI') & (bmw['model'].str.contains('GRANDES', case=False)), 'car_class'] = 'MPV'   
bmw.loc[(bmw['car'] == 'MITSUBISHI') & (bmw['model'].str.contains('DELICA', case=False)), 'car_class'] = 'MPV'        
bmw.loc[(bmw['car'] == 'MITSUBISHI') & (bmw['model'].str.contains('COLT', case=False)), 'car_class'] = 'B'         
bmw.loc[(bmw['car'] == 'MITSUBISHI') & (bmw['model'].str.contains('GALANT', case=False)), 'car_class'] = 'C'
bmw.loc[(bmw['car'] == 'MITSUBISHI') & (bmw['model'].str.contains('3000 GT', case=False)), 'car_class'] = 'S'
bmw.loc[(bmw['car'] == 'MITSUBISHI') & (bmw['model'].str.contains('ASX', case=False)), 'car_class'] = 'M SUV'
bmw.loc[(bmw['car'] == 'MITSUBISHI') & (bmw['model'].str.contains('RVR', case=False)), 'car_class'] = 'S SUV'

bmw.loc[(bmw['car'] == 'HONDA') & (bmw['model'].str.contains('CR V', case=False)), 'car_class'] = 'S SUV'
bmw.loc[(bmw['car'] == 'HONDA') & (bmw['model'].str.contains('CR-V', case=False)), 'car_class'] = 'S SUV'
bmw.loc[(bmw['car'] == 'HONDA') & (bmw['model'].str.contains('HR-V', case=False)), 'car_class'] = 'S SUV'
bmw.loc[(bmw['car'] == 'HONDA') & (bmw['model'].str.contains('ACCORD', case=False)), 'car_class'] = 'D'
bmw.loc[(bmw['car'] == 'HONDA') & (bmw['model'].str.contains('CIVIC', case=False)), 'car_class'] = 'C'
bmw.loc[(bmw['car'] == 'HONDA') & (bmw['model'].str.contains('ELYSION', case=False)), 'car_class'] = 'MPV'
bmw.loc[(bmw['car'] == 'HONDA') & (bmw['model'].str.contains('FIT', case=False)), 'car_class'] = 'B'
bmw.loc[(bmw['car'] == 'HONDA') & (bmw['model'].str.contains('ODYSSEY', case=False)), 'car_class'] = 'MPV'
bmw.loc[(bmw['car'] == 'HONDA') & (bmw['model'].str.contains('PILOT', case=False)), 'car_class'] = 'M SUV' 
bmw.loc[(bmw['car'] == 'HONDA') & (bmw['model'].str.contains('PASSPORT', case=False)), 'car_class'] = 'M SUV'     
bmw.loc[(bmw['car'] == 'HONDA') & (bmw['model'].str.contains('JAZZ', case=False)), 'car_class'] = 'B'    
bmw.loc[(bmw['car'] == 'HONDA') & (bmw['model'].str.contains('EDIX', case=False)), 'car_class'] = 'MPV'    
bmw.loc[(bmw['car'] == 'HONDA') & (bmw['model'].str.contains('STEP WAGON', case=False)), 'car_class'] = 'MPV'         
bmw.loc[(bmw['car'] == 'HONDA') & (bmw['model'].str.contains('INSPIRE', case=False)), 'car_class'] = 'D'
bmw.loc[(bmw['car'] == 'HONDA') & (bmw['model'].str.contains('ELEMENT', case=False)), 'car_class'] = 'S SUV' 

bmw.loc[(bmw['car'] == 'HYUNDAI') & (bmw['model'].str.contains('ELANTRA', case=False)), 'car_class'] = 'C'
bmw.loc[(bmw['car'] == 'HYUNDAI') & (bmw['model'].str.contains('SONATA', case=False)), 'car_class'] = 'D'
bmw.loc[(bmw['car'] == 'HYUNDAI') & (bmw['model'].str.contains('TUCSON', case=False)), 'car_class'] = 'S SUV'
bmw.loc[(bmw['car'] == 'HYUNDAI') & (bmw['model'].str.contains('SANTA', case=False)), 'car_class'] = 'S SUV'
bmw.loc[(bmw['car'] == 'HYUNDAI') & (bmw['model'].str.contains('SOLARIS', case=False)), 'car_class'] = 'C'   
bmw.loc[(bmw['car'] == 'HYUNDAI') & (bmw['model'].str.contains('VELOSTER', case=False)), 'car_class'] = 'C'       
bmw.loc[(bmw['car'] == 'HYUNDAI') & (bmw['model'].str.contains('SANTAFE', case=False)), 'car_class'] = 'S SUV'    
bmw.loc[(bmw['car'] == 'HYUNDAI') & (bmw['model'].str.contains('ACCENT', case=False)), 'car_class'] = 'B'         
bmw.loc[(bmw['car'] == 'HYUNDAI') & (bmw['model'].str.contains('TIBURON', case=False)), 'car_class'] = 'S'  
bmw.loc[(bmw['car'] == 'HYUNDAI') & (bmw['model'].str.contains('LANTRA', case=False)), 'car_class'] = 'C'
bmw.loc[(bmw['car'] == 'HYUNDAI') & (bmw['model'].str.contains('AZERA', case=False)), 'car_class'] = 'E'
bmw.loc[(bmw['car'] == 'HYUNDAI') & (bmw['model'].str.contains('AVANTE', case=False)), 'car_class'] = 'C' 
bmw.loc[(bmw['car'] == 'HYUNDAI') & (bmw['model'].str.contains('GALLOPER', case=False)), 'car_class'] = 'L SUV' 
bmw.loc[(bmw['car'] == 'HYUNDAI') & (bmw['model'].str.contains('VERACRUZ', case=False)), 'car_class'] = 'M SUV'      

bmw.loc[(bmw['car'] == 'LEXUS') & (bmw['model'].str.contains('LX', case=False)), 'car_class'] = 'L SUV'
bmw.loc[(bmw['car'] == 'LEXUS') & (bmw['model'].str.contains('RX', case=False)), 'car_class'] = 'M SUV'
bmw.loc[(bmw['car'] == 'LEXUS') & (bmw['model'].str.contains('GX', case=False)), 'car_class'] = 'L SUV'
bmw.loc[(bmw['car'] == 'LEXUS') & (bmw['model'].str.contains('GS', case=False)), 'car_class'] = 'E'
bmw.loc[(bmw['car'] == 'LEXUS') & (bmw['model'].str.contains('LS', case=False)), 'car_class'] = 'E'
bmw.loc[(bmw['car'] == 'LEXUS') & (bmw['model'].str.contains('IS', case=False)), 'car_class'] = 'D'
bmw.loc[(bmw['car'] == 'LEXUS') & (bmw['model'].str.contains('NX', case=False)), 'car_class'] = 'S SUV'
bmw.loc[(bmw['car'] == 'LEXUS') & (bmw['model'].str.contains('ES', case=False)), 'car_class'] = 'E'

bmw.loc[(bmw['car'] == 'KIA') & (bmw['model'].str.contains('SORENTO', case=False)), 'car_class'] = 'S SUV'
bmw.loc[(bmw['car'] == 'KIA') & (bmw['model'].str.contains('FORTE', case=False)), 'car_class'] = 'C'
bmw.loc[(bmw['car'] == 'KIA') & (bmw['model'].str.contains('OPTIMA', case=False)), 'car_class'] = 'D'
bmw.loc[(bmw['car'] == 'KIA') & (bmw['model'].str.contains('RIO', case=False)), 'car_class'] = 'B'
bmw.loc[(bmw['car'] == 'KIA') & (bmw['model'].str.contains('SPORTAGE', case=False)), 'car_class'] = 'S SUV'
bmw.loc[(bmw['car'] == 'KIA') & (bmw['model'].str.contains('CERATO', case=False)), 'car_class'] = 'C'
bmw.loc[(bmw['car'] == 'KIA') & (bmw['model'].str.contains('K5', case=False)), 'car_class'] = 'D'
bmw.loc[(bmw['car'] == 'KIA') & (bmw['model'].str.contains('PICANTO', case=False)), 'car_class'] = 'A'    
bmw.loc[(bmw['car'] == 'KIA') & (bmw['model'].str.contains('CARNIVAL', case=False)), 'car_class'] = 'MPV'     
bmw.loc[(bmw['car'] == 'KIA') & (bmw['model'].str.contains('SOUL', case=False)), 'car_class'] = 'B'       

bmw.loc[(bmw['car'] == 'FORD') & (bmw['model'].str.contains('FUSION', case=False)), 'car_class'] = 'D'    
bmw.loc[(bmw['car'] == 'FORD') & (bmw['model'].str.contains('FOCUS', case=False)), 'car_class'] = 'C'  
bmw.loc[(bmw['car'] == 'FORD') & (bmw['model'].str.contains('MONDEO', case=False)), 'car_class'] = 'D'    
bmw.loc[(bmw['car'] == 'FORD') & (bmw['model'].str.contains('ESCAPE', case=False)), 'car_class'] = 'S SUV'
bmw.loc[(bmw['car'] == 'FORD') & (bmw['model'].str.contains('EXPLORER', case=False)), 'car_class'] = 'S SUV'  
bmw.loc[(bmw['car'] == 'FORD') & (bmw['model'].str.contains('TRANSIT', case=False)), 'car_class'] = 'MPV'  
bmw.loc[(bmw['car'] == 'FORD') & (bmw['model'].str.contains('FIESTA', case=False)), 'car_class'] = 'B'  
bmw.loc[(bmw['car'] == 'FORD') & (bmw['model'].str.contains('ESCORT', case=False)), 'car_class'] = 'C'  
bmw.loc[(bmw['car'] == 'FORD') & (bmw['model'].str.contains('MUSTANG', case=False)), 'car_class'] = 'S'  
bmw.loc[(bmw['car'] == 'FORD') & (bmw['model'].str.contains('BED BLITZ', case=False)), 'car_class'] = 'M'  
bmw.loc[(bmw['car'] == 'FORD') & (bmw['model'].str.contains('SCORPIO', case=False)), 'car_class'] = 'E'   
bmw.loc[(bmw['car'] == 'FORD') & (bmw['model'].str.contains('EXPEDITION', case=False)), 'car_class'] = 'L SUV' 
bmw.loc[(bmw['car'] == 'FORD') & (bmw['model'].str.contains('GALAXY', case=False)), 'car_class'] = 'M'
bmw.loc[(bmw['car'] == 'FORD') & (bmw['model'].str.contains('EXCURSION', case=False)), 'car_class'] = 'L SUV' 
bmw.loc[(bmw['car'] == 'FORD') & (bmw['model'].str.contains('SIERRA', case=False)), 'car_class'] = 'D' 
bmw.loc[(bmw['car'] == 'FORD') & (bmw['model'].str.contains('TAURUS', case=False)), 'car_class'] = 'D' 
bmw.loc[(bmw['car'] == 'FORD') & (bmw['model'].str.contains('WINDSTAR', case=False)), 'car_class'] = 'MPV'      

bmw.loc[(bmw['car'] == 'MAZDA') & (bmw['model'].str.contains('2', case=False)), 'car_class'] = 'B' 
bmw.loc[(bmw['car'] == 'MAZDA') & (bmw['model'].str.contains('3', case=False)), 'car_class'] = 'C'    
bmw.loc[(bmw['car'] == 'MAZDA') & (bmw['model'].str.contains('6', case=False)), 'car_class'] = 'D'  
bmw.loc[(bmw['car'] == 'MAZDA') & (bmw['model'].str.contains('PREMACY', case=False)), 'car_class'] = 'MPV' 
bmw.loc[(bmw['car'] == 'MAZDA') & (bmw['model'].str.contains('MPV', case=False)), 'car_class'] = 'MPV'  
bmw.loc[(bmw['car'] == 'MAZDA') & (bmw['model'].str.contains('RX8', case=False)), 'car_class'] = 'S'  
bmw.loc[(bmw['car'] == 'MAZDA') & (bmw['model'].str.contains('CX-5', case=False)), 'car_class'] = 'S SUV' 
bmw.loc[(bmw['car'] == 'MAZDA') & (bmw['model'].str.contains('CX 5', case=False)), 'car_class'] = 'S SUV' 
bmw.loc[(bmw['car'] == 'MAZDA') & (bmw['model'].str.contains('CX 7', case=False)), 'car_class'] = 'M SUV'
bmw.loc[(bmw['car'] == 'MAZDA') & (bmw['model'].str.contains('CX-7', case=False)), 'car_class'] = 'M SUV'  
bmw.loc[(bmw['car'] == 'MAZDA') & (bmw['model'].str.contains('CX9', case=False)), 'car_class'] = 'M SUV' 
bmw.loc[(bmw['car'] == 'MAZDA') & (bmw['model'].str.contains('CX-9', case=False)), 'car_class'] = 'M SUV'
bmw.loc[(bmw['car'] == 'MAZDA') & (bmw['model'].str.contains('CX 9', case=False)), 'car_class'] = 'M SUV'
bmw.loc[(bmw['car'] == 'MAZDA') & (bmw['model'].str.contains('VERISA', case=False)), 'car_class'] = 'B' 
bmw.loc[(bmw['car'] == 'MAZDA') & (bmw['model'].str.contains('MILLENIA', case=False)), 'car_class'] = 'E' 
bmw.loc[(bmw['car'] == 'MAZDA') & (bmw['model'].str.contains('DEMIO', case=False)), 'car_class'] = 'B'
bmw.loc[(bmw['car'] == 'MAZDA') & (bmw['model'].str.contains('TRIBUTE', case=False)), 'car_class'] = 'S SUV'
bmw.loc[(bmw['car'] == 'MAZDA') & (bmw['model'].str.contains('CAPELLA', case=False)), 'car_class'] = 'D'  

bmw.loc[(bmw['car'] == 'AUDI') & (bmw['model'].str.contains('A4', case=False)), 'car_class'] = 'D' 
bmw.loc[(bmw['car'] == 'AUDI') & (bmw['model'].str.contains('S4', case=False)), 'car_class'] = 'D' 
bmw.loc[(bmw['car'] == 'AUDI') & (bmw['model'].str.contains('A 4', case=False)), 'car_class'] = 'D'
bmw.loc[(bmw['car'] == 'AUDI') & (bmw['model'].str.contains('A5', case=False)), 'car_class'] = 'D' 
bmw.loc[(bmw['car'] == 'AUDI') & (bmw['model'].str.contains('A3', case=False)), 'car_class'] = 'C' 
bmw.loc[(bmw['car'] == 'AUDI') & (bmw['model'].str.contains('A6', case=False)), 'car_class'] = 'E' 
bmw.loc[(bmw['car'] == 'AUDI') & (bmw['model'].str.contains('80', case=False)), 'car_class'] = 'D' 
bmw.loc[(bmw['car'] == 'AUDI') & (bmw['model'].str.contains('Q7', case=False)), 'car_class'] = 'M SUV' 
bmw.loc[(bmw['car'] == 'AUDI') & (bmw['model'].str.contains('TT', case=False)), 'car_class'] = 'S' 
bmw.loc[(bmw['car'] == 'AUDI') & (bmw['model'].str.contains('Q5', case=False)), 'car_class'] = 'M SUV' 
bmw.loc[(bmw['car'] == 'AUDI') & (bmw['model'].str.contains('Q3', case=False)), 'car_class'] = 'S SUV' 
bmw.loc[(bmw['car'] == 'AUDI') & (bmw['model'].str.contains('100', case=False)), 'car_class'] = 'E' 
bmw.loc[(bmw['car'] == 'AUDI') & (bmw['model'].str.contains('A8', case=False)), 'car_class'] = 'F' 

bmw.loc[(bmw['car'] == 'INFINITI') & (bmw['model'].str.contains('FX', case=False)), 'car_class'] = 'M SUV' 
bmw.loc[(bmw['car'] == 'INFINITI') & (bmw['model'].str.contains('FX37', case=False)), 'car_class'] = 'M SUV' 
bmw.loc[(bmw['car'] == 'INFINITI') & (bmw['model'].str.contains('M', case=False)), 'car_class'] = 'E'          
bmw.loc[(bmw['car'] == 'INFINITI') & (bmw['model'].str.contains('QX', case=False)), 'car_class'] = 'M SUV' 
bmw.loc[(bmw['car'] == 'INFINITI') & (bmw['model'].str.contains('EX', case=False)), 'car_class'] = 'M SUV' 
bmw.loc[(bmw['car'] == 'INFINITI') & (bmw['model'].str.contains('QX4', case=False)), 'car_class'] = 'M SUV' 
bmw.loc[(bmw['car'] == 'INFINITI') & (bmw['model'].str.contains('G 35', case=False)), 'car_class'] = 'D'
bmw.loc[(bmw['car'] == 'INFINITI') & (bmw['model'].str.contains('G 37', case=False)), 'car_class'] = 'D'
bmw.loc[(bmw['car'] == 'INFINITI') & (bmw['model'].str.contains('G', case=False)), 'car_class'] = 'D'

bmw.loc[(bmw['car'] == 'RENAULT') & (bmw['model'].str.contains('LOGAN', case=False)), 'car_class'] = 'B'
bmw.loc[(bmw['car'] == 'RENAULT') & (bmw['model'].str.contains('CLIO', case=False)), 'car_class'] = 'B'
bmw.loc[(bmw['car'] == 'RENAULT') & (bmw['model'].str.contains('MEGANE', case=False)), 'car_class'] = 'C'
bmw.loc[(bmw['car'] == 'RENAULT') & (bmw['model'].str.contains('LAGUNA', case=False)), 'car_class'] = 'C'
bmw.loc[(bmw['car'] == 'RENAULT') & (bmw['model'].str.contains('TWINGO', case=False)), 'car_class'] = 'A'
bmw.loc[(bmw['car'] == 'RENAULT') & (bmw['model'].str.contains('DUSTER', case=False)), 'car_class'] = 'C' 
bmw.loc[(bmw['car'] == 'RENAULT') & (bmw['model'].str.contains('KANGOO ', case=False)), 'car_class'] = 'M'
bmw.loc[(bmw['car'] == 'RENAULT') & (bmw['model'].str.contains('SANDERO', case=False)), 'car_class'] = 'S SUV'
bmw.loc[(bmw['car'] == 'RENAULT') & (bmw['model'].str.contains('ESPACE', case=False)), 'car_class'] = 'M' 
bmw.loc[(bmw['car'] == 'RENAULT') & (bmw['model'].str.contains('SAFRANE', case=False)), 'car_class'] = 'E'         

bmw.loc[(bmw['car'] == 'CHEVROLET') & (bmw['model'].str.contains('NIVA', case=False)), 'car_class'] = 'S SUV'
bmw.loc[(bmw['car'] == 'CHEVROLET') & (bmw['model'].str.contains('CRUZE', case=False)), 'car_class'] = 'C'
bmw.loc[(bmw['car'] == 'CHEVROLET') & (bmw['model'].str.contains('AVEO', case=False)), 'car_class'] = 'B'
bmw.loc[(bmw['car'] == 'CHEVROLET') & (bmw['model'].str.contains('CAMARO', case=False)), 'car_class'] = 'S'
bmw.loc[(bmw['car'] == 'CHEVROLET') & (bmw['model'].str.contains('TRAILBLAZER', case=False)), 'car_class'] = 'M SUV'
bmw.loc[(bmw['car'] == 'CHEVROLET') & (bmw['model'].str.contains('CAPTIVA', case=False)), 'car_class'] = 'S SUV'   
bmw.loc[(bmw['car'] == 'CHEVROLET') & (bmw['model'].str.contains('LACETTI', case=False)), 'car_class'] = 'C'    
bmw.loc[(bmw['car'] == 'CHEVROLET') & (bmw['model'].str.contains('LANOS', case=False)), 'car_class'] = 'B'      
bmw.loc[(bmw['car'] == 'CHEVROLET') & (bmw['model'].str.contains('EQUINOX', case=False)), 'car_class'] = 'M SUV'
bmw.loc[(bmw['car'] == 'CHEVROLET') & (bmw['model'].str.contains('MALIBU', case=False)), 'car_class'] = 'D'
bmw.loc[(bmw['car'] == 'CHEVROLET') & (bmw['model'].str.contains('BLAZER', case=False)), 'car_class'] = 'M SUV'  
bmw.loc[(bmw['car'] == 'CHEVROLET') & (bmw['model'].str.contains('ALERO', case=False)), 'car_class'] = 'D'    
bmw.loc[(bmw['car'] == 'CHEVROLET') & (bmw['model'].str.contains('TRACKER', case=False)), 'car_class'] = 'S SUV'     
bmw.loc[(bmw['car'] == 'CHEVROLET') & (bmw['model'].str.contains('VOLT', case=False)), 'car_class'] = 'C'         

bmw.loc[(bmw['car'] == 'SUZUKI') & (bmw['model'].str.contains('SX4', case=False)), 'car_class'] = 'B'        
bmw.loc[(bmw['car'] == 'SUZUKI') & (bmw['model'].str.contains('VITARA', case=False)), 'car_class'] = 'S SUV'        
bmw.loc[(bmw['car'] == 'SUZUKI') & (bmw['model'].str.contains('SWIFT', case=False)), 'car_class'] = 'B'       
bmw.loc[(bmw['car'] == 'SUZUKI') & (bmw['model'].str.contains('KIZASHI', case=False)), 'car_class'] = 'D'   
bmw.loc[(bmw['car'] == 'SUZUKI') & (bmw['model'].str.contains('XL7', case=False)), 'car_class'] = 'M SUV'     
bmw.loc[(bmw['car'] == 'SUZUKI') & (bmw['model'].str.contains('AERIO', case=False)), 'car_class'] = 'A'     
bmw.loc[(bmw['car'] == 'SUZUKI') & (bmw['model'].str.contains('WAGON', case=False)), 'car_class'] = 'A'     
bmw.loc[(bmw['car'] == 'SUZUKI') & (bmw['model'].str.contains('ESCUDO', case=False)), 'car_class'] = 'S SUV'                
bmw.loc[(bmw['car'] == 'SUZUKI') & (bmw['model'].str.contains('SPLASH', case=False)), 'car_class'] = 'A'     
bmw.loc[(bmw['car'] == 'SUZUKI') & (bmw['model'].str.contains('ALTO', case=False)), 'car_class'] = 'A'          
bmw.loc[(bmw['car'] == 'SUZUKI') & (bmw['model'].str.contains('CULTUS', case=False)), 'car_class'] = 'A'         
bmw.loc[(bmw['car'] == 'SUZUKI') & (bmw['model'].str.contains('APV', case=False)), 'car_class'] = 'MPV'           
bmw.loc[(bmw['car'] == 'SUZUKI') & (bmw['model'].str.contains('LIANA', case=False)), 'car_class'] = 'A'   
bmw.loc[(bmw['car'] == 'SUZUKI') & (bmw['model'].str.contains('JIMNY', case=False)), 'car_class'] = 'B'      
bmw.loc[(bmw['car'] == 'SUZUKI') & (bmw['model'].str.contains('VAN', case=False)), 'car_class'] = 'MPV'         

bmw.loc[(bmw['car'] == 'JEEP') & (bmw['model'].str.contains('CHEROKEE', case=False)), 'car_class'] = 'M SUV'  
bmw.loc[(bmw['car'] == 'JEEP') & (bmw['model'].str.contains('LIBERTY', case=False)), 'car_class'] = 'S SUV'  
bmw.loc[(bmw['car'] == 'JEEP') & (bmw['model'].str.contains('WRANGLER', case=False)), 'car_class'] = 'S SUV'
bmw.loc[(bmw['car'] == 'JEEP') & (bmw['model'].str.contains('PATRIOT', case=False)), 'car_class'] = 'S SUV'  
bmw.loc[(bmw['car'] == 'JEEP') & (bmw['model'].str.contains('COMPASS', case=False)), 'car_class'] = 'S SUV'  
bmw.loc[(bmw['car'] == 'JEEP') & (bmw['model'].str.contains('COMMANDER', case=False)), 'car_class'] = 'S SUV' 

bmw.loc[(bmw['car'] == 'SUBARU') & (bmw['model'].str.contains('FORESTER', case=False)), 'car_class'] = 'S SUV' 
bmw.loc[(bmw['car'] == 'SUBARU') & (bmw['model'].str.contains('LEGACY', case=False)), 'car_class'] = 'D' 
bmw.loc[(bmw['car'] == 'SUBARU') & (bmw['model'].str.contains('IMPREZA', case=False)), 'car_class'] = 'C' 
bmw.loc[(bmw['car'] == 'SUBARU') & (bmw['model'].str.contains('OUTBACK', case=False)), 'car_class'] = 'D' 
bmw.loc[(bmw['car'] == 'SUBARU') & (bmw['model'].str.contains('TRAVIQ', case=False)), 'car_class'] = 'MPV' 
bmw.loc[(bmw['car'] == 'SUBARU') & (bmw['model'].str.contains('TRIBECA', case=False)), 'car_class'] = 'M SUV' 

bmw.loc[(bmw['car'] == 'PORSCHE') & (bmw['model'].str.contains('CAYENNE', case=False)), 'car_class'] = 'M SUV' 
bmw.loc[(bmw['car'] == 'PORSCHE') & (bmw['model'].str.contains('PANAMERA', case=False)), 'car_class'] = 'E' 
bmw.loc[(bmw['car'] == 'PORSCHE') & (bmw['model'].str.contains('MACAN', case=False)), 'car_class'] = 'S SUV' 
bmw.loc[(bmw['car'] == 'PORSCHE') & (bmw['model'].str.contains('CAYMAN', case=False)), 'car_class'] = 'S' 
bmw.loc[(bmw['car'] == 'PORSCHE') & (bmw['model'].str.contains('CARRERA', case=False)), 'car_class'] = 'S' 
bmw.loc[(bmw['car'] == 'PORSCHE') & (bmw['model'].str.contains('BOXSTER', case=False)), 'car_class'] = 'S' 

bmw.loc[(bmw['car'] == 'PEUGEOT') & (bmw['model'].str.contains('206', case=False)), 'car_class'] = 'B' 
bmw.loc[(bmw['car'] == 'PEUGEOT') & (bmw['model'].str.contains('207', case=False)), 'car_class'] = 'B' 
bmw.loc[(bmw['car'] == 'PEUGEOT') & (bmw['model'].str.contains('307', case=False)), 'car_class'] = 'C'
bmw.loc[(bmw['car'] == 'PEUGEOT') & (bmw['model'].str.contains('308', case=False)), 'car_class'] = 'C'
bmw.loc[(bmw['car'] == 'PEUGEOT') & (bmw['model'].str.contains('PARTNER', case=False)), 'car_class'] = 'M'
bmw.loc[(bmw['car'] == 'PEUGEOT') & (bmw['model'].str.contains('806', case=False)), 'car_class'] = 'MPV'
bmw.loc[(bmw['car'] == 'PEUGEOT') & (bmw['model'].str.contains('607', case=False)), 'car_class'] = 'E'
bmw.loc[(bmw['car'] == 'PEUGEOT') & (bmw['model'].str.contains('605', case=False)), 'car_class'] = 'E' 
bmw.loc[(bmw['car'] == 'PEUGEOT') & (bmw['model'].str.contains('405', case=False)), 'car_class'] = 'D' 
bmw.loc[(bmw['car'] == 'PEUGEOT') & (bmw['model'].str.contains('407', case=False)), 'car_class'] = 'D' 
bmw.loc[(bmw['car'] == 'PEUGEOT') & (bmw['model'].str.contains('406', case=False)), 'car_class'] = 'D' 
bmw.loc[(bmw['car'] == 'PEUGEOT') & (bmw['model'].str.contains('508', case=False)), 'car_class'] = 'D' 
bmw.loc[(bmw['car'] == 'PEUGEOT') & (bmw['model'].str.contains('1007', case=False)), 'car_class'] = 'A' 
bmw.loc[(bmw['car'] == 'PEUGEOT') & (bmw['model'].str.contains('3008', case=False)), 'car_class'] = 'S SUV' 

bmw.loc[(bmw['car'] == 'SKODA') & (bmw['model'].str.contains('FABIA', case=False)), 'car_class'] = 'B' 
bmw.loc[(bmw['car'] == 'SKODA') & (bmw['model'].str.contains('OCTAVIA', case=False)), 'car_class'] = 'C' 
bmw.loc[(bmw['car'] == 'SKODA') & (bmw['model'].str.contains('RAPID', case=False)), 'car_class'] = 'C'
bmw.loc[(bmw['car'] == 'SKODA') & (bmw['model'].str.contains('SUPER', case=False)), 'car_class'] = 'D'
bmw.loc[(bmw['car'] == 'SKODA') & (bmw['model'].str.contains('FORMAN', case=False)), 'car_class'] = 'C'
bmw.loc[(bmw['car'] == 'SKODA') & (bmw['model'].str.contains('ROOMSTER', case=False)), 'car_class'] = 'C'

bmw.loc[(bmw['car'] == 'RANGE ROVER'), 'car_class'] = 'M SUV'
bmw.loc[(bmw['car'] == 'RANGE ROVER') & (bmw['model'].str.contains('SPORT', case=False)), 'car_class'] = 'M SUV'
bmw.loc[(bmw['car'] == 'RANGE ROVER') & (bmw['model'].str.contains('EVOQUE', case=False)), 'car_class'] = 'S SUV'
bmw.loc[(bmw['car'] == 'RANGE ROVER') & (bmw['model'].str.contains('VELAR', case=False)), 'car_class'] = 'M SUV'
bmw.loc[(bmw['car'] == 'RANGE ROVER') & (bmw['model'].str.contains('VOGUE', case=False)), 'car_class'] = 'L SUV'

bmw.loc[(bmw['car'] == 'DODGE'), 'car_class'] = 'D'
bmw.loc[(bmw['car'] == 'DODGE') & (bmw['model'].str.contains('CARAVAN', case=False)), 'car_class'] = 'MPV'
bmw.loc[(bmw['car'] == 'DODGE') & (bmw['model'].str.contains('NEON', case=False)), 'car_class'] = 'C'
bmw.loc[(bmw['car'] == 'DODGE') & (bmw['model'].str.contains('DART', case=False)), 'car_class'] = 'C'
bmw.loc[(bmw['car'] == 'DODGE') & (bmw['model'].str.contains('DURANGO', case=False)), 'car_class'] = 'M SUV'
bmw.loc[(bmw['car'] == 'DODGE') & (bmw['model'].str.contains('CALIBER', case=False)), 'car_class'] = 'MPV'
bmw.loc[(bmw['car'] == 'DODGE') & (bmw['model'].str.contains('CHARGER', case=False)), 'car_class'] = 'S'
bmw.loc[(bmw['car'] == 'DODGE') & (bmw['model'].str.contains('CHALLENGER', case=False)), 'car_class'] = 'S'
bmw.loc[(bmw['car'] == 'DODGE') & (bmw['model'].str.contains('STRATUS', case=False)), 'car_class'] = 'D'
bmw.loc[(bmw['car'] == 'DODGE') & (bmw['model'].str.contains('SPIRIT', case=False)), 'car_class'] = 'D'
bmw.loc[(bmw['car'] == 'DODGE') & (bmw['model'].str.contains('MAGNUM', case=False)), 'car_class'] = 'D'
bmw.loc[(bmw['car'] == 'DODGE') & (bmw['model'].str.contains('AVENGER', case=False)), 'car_class'] = 'D'

bmw.loc[(bmw['car'] == 'CHRYSLER'), 'car_class'] = 'E'
bmw.loc[(bmw['car'] == 'CHRYSLER') & (bmw['model'].str.contains('300', case=False)), 'car_class'] = 'E'
bmw.loc[(bmw['car'] == 'CHRYSLER') & (bmw['model'].str.contains('VOYAGER', case=False)), 'car_class'] = 'MPV'
bmw.loc[(bmw['car'] == 'CHRYSLER') & (bmw['model'].str.contains('CARAVAN', case=False)), 'car_class'] = 'MPV'
bmw.loc[(bmw['car'] == 'CHRYSLER') & (bmw['model'].str.contains('TOWN', case=False)), 'car_class'] = 'MPV'
bmw.loc[(bmw['car'] == 'CHRYSLER') & (bmw['model'].str.contains('200', case=False)), 'car_class'] = 'D'
bmw.loc[(bmw['car'] == 'CHRYSLER') & (bmw['model'].str.contains('STRATUS', case=False)), 'car_class'] = 'D'
bmw.loc[(bmw['car'] == 'CHRYSLER') & (bmw['model'].str.contains('NEON', case=False)), 'car_class'] = 'C'
bmw.loc[(bmw['car'] == 'CHRYSLER') & (bmw['model'].str.contains('CRUISER', case=False)), 'car_class'] = 'C'
bmw.loc[(bmw['car'] == 'CHRYSLER') & (bmw['model'].str.contains('SEBRING', case=False)), 'car_class'] = 'D'
bmw.loc[(bmw['car'] == 'CHRYSLER') & (bmw['model'].str.contains('CROSSFIRE', case=False)), 'car_class'] = 'S'
bmw.loc[(bmw['car'] == 'CHRYSLER') & (bmw['model'].str.contains('PACIFICA', case=False)), 'car_class'] = 'MPV'

bmw.loc[(bmw['car'] == 'VOLVO'), 'car_class'] = 'D'
bmw.loc[(bmw['car'] == 'VOLVO') & (bmw['model'].str.contains('XC90', case=False)), 'car_class'] = 'M SUV'
bmw.loc[(bmw['car'] == 'VOLVO') & (bmw['model'].str.contains('XC 90', case=False)), 'car_class'] = 'M SUV'
bmw.loc[(bmw['car'] == 'VOLVO') & (bmw['model'].str.contains('XC70', case=False)), 'car_class'] = 'E'
bmw.loc[(bmw['car'] == 'VOLVO') & (bmw['model'].str.contains('940', case=False)), 'car_class'] = 'E'
bmw.loc[(bmw['car'] == 'VOLVO') & (bmw['model'].str.contains('960', case=False)), 'car_class'] = 'E'
bmw.loc[(bmw['car'] == 'VOLVO') & (bmw['model'].str.contains('S60', case=False)), 'car_class'] = 'D'
bmw.loc[(bmw['car'] == 'VOLVO') & (bmw['model'].str.contains('S 60', case=False)), 'car_class'] = 'D'
bmw.loc[(bmw['car'] == 'VOLVO') & (bmw['model'].str.contains('S 70', case=False)), 'car_class'] = 'D'
bmw.loc[(bmw['car'] == 'VOLVO') & (bmw['model'].str.contains('S 80', case=False)), 'car_class'] = 'E'
bmw.loc[(bmw['car'] == 'VOLVO') & (bmw['model'].str.contains('S80', case=False)), 'car_class'] = 'E'
bmw.loc[(bmw['car'] == 'VOLVO') & (bmw['model'].str.contains('S40', case=False)), 'car_class'] = 'D'
bmw.loc[(bmw['car'] == 'VOLVO') & (bmw['model'].str.contains('S 40', case=False)), 'car_class'] = 'D'
bmw.loc[(bmw['car'] == 'VOLVO') & (bmw['model'].str.contains('V 40', case=False)), 'car_class'] = 'C'
bmw.loc[(bmw['car'] == 'VOLVO') & (bmw['model'].str.contains('V40', case=False)), 'car_class'] = 'C'

bmw.loc[(bmw['car'] == 'SEAT'), 'car_class'] = 'B'
bmw.loc[(bmw['car'] == 'SEAT') & (bmw['model'].str.contains('TOLEDO', case=False)), 'car_class'] = 'C'
bmw.loc[(bmw['car'] == 'SEAT') & (bmw['model'].str.contains('IBIZA', case=False)), 'car_class'] = 'B'
bmw.loc[(bmw['car'] == 'SEAT') & (bmw['model'].str.contains('CORDOBA', case=False)), 'car_class'] = 'B'

bmw.loc[(bmw['car'] == 'VAUXHALL') & (bmw['model'].str.contains('ASTRA', case=False)), 'car_class'] = 'C'
bmw.loc[(bmw['car'] == 'VAUXHALL') & (bmw['model'].str.contains('VECTRA', case=False)), 'car_class'] = 'D'
bmw.loc[(bmw['car'] == 'VAUXHALL') & (bmw['model'].str.contains('ZAFIRA', case=False)), 'car_class'] = 'MPV'

bmw.loc[(bmw['car'] == 'ISUZU'), 'car_class'] = 'M SUV'
bmw.loc[(bmw['car'] == 'ISUZU') & (bmw['model'].str.contains('RODEO', case=False)), 'car_class'] = 'M SUV'
bmw.loc[(bmw['car'] == 'ISUZU') & (bmw['model'].str.contains('TROOPER', case=False)), 'car_class'] = 'M SUV'
bmw.loc[(bmw['car'] == 'ISUZU') & (bmw['model'].str.contains('AXIOM', case=False)), 'car_class'] = 'MPV'
bmw.loc[(bmw['car'] == 'ISUZU') & (bmw['model'].str.contains('AMIGO', case=False)), 'car_class'] = 'S SUV'

bmw.loc[(bmw['car'] == 'SATURN'), 'car_class'] = 'C'
bmw.loc[(bmw['car'] == 'SATURN') & (bmw['model'].str.contains('VUE', case=False)), 'car_class'] = 'M SUV'

bmw.loc[(bmw['car'] == 'CITROEN'), 'car_class'] = 'C'
bmw.loc[(bmw['car'] == 'CITROEN') & (bmw['model'].str.contains('BERLINGO', case=False)), 'car_class'] = 'M'
bmw.loc[(bmw['car'] == 'CITROEN') & (bmw['model'].str.contains('JUMPER', case=False)), 'car_class'] = 'MPV'

bmw.loc[(bmw['car'] == 'ACURA') & (bmw['model'].str.contains('MDX', case=False)), 'car_class'] = 'M SUV'
bmw.loc[(bmw['car'] == 'ACURA') & (bmw['model'].str.contains('ILX', case=False)), 'car_class'] = 'C'
bmw.loc[(bmw['car'] == 'ACURA') & (bmw['model'].str.contains('TSX', case=False)), 'car_class'] = 'D'
bmw.loc[(bmw['car'] == 'ACURA') & (bmw['model'].str.contains('TL', case=False)), 'car_class'] = 'E'

bmw.loc[(bmw['car'] == 'LINCOLN') & (bmw['model'].str.contains('NAVIGATOR', case=False)), 'car_class'] = 'L SUV'
bmw.loc[(bmw['car'] == 'LINCOLN') & (bmw['model'].str.contains('TOWN CAR', case=False)), 'car_class'] = 'F'
bmw.loc[(bmw['car'] == 'LINCOLN') & (bmw['model'].str.contains('AVIATOR', case=False)), 'car_class'] = 'M SUV'
bmw.loc[(bmw['car'] == 'LINCOLN') & (bmw['model'].str.contains('CONTINENTAL', case=False)), 'car_class'] = 'F'

bmw.loc[(bmw['car'] == 'CADILLAC'), 'car_class'] = 'E'
bmw.loc[(bmw['car'] == 'CADILLAC') & (bmw['model'].str.contains('ESCALADE', case=False)), 'car_class'] = 'L SUV'
bmw.loc[(bmw['car'] == 'CADILLAC') & (bmw['model'].str.contains('SRX', case=False)), 'car_class'] = 'M SUV'
bmw.loc[(bmw['car'] == 'CADILLAC') & (bmw['model'].str.contains('FLEETWOOD', case=False)), 'car_class'] = 'F'

bmw.loc[(bmw['car'] == 'ALFA ROMEO'), 'car_class'] = 'C'
bmw.loc[(bmw['car'] == 'HUMMER'), 'car_class'] = 'M SUV'
bmw.loc[(bmw['car'] == 'DAIHATSU'), 'car_class'] = 'B'
bmw.loc[(bmw['car'] == 'SAMAND'), 'car_class'] = 'D'
bmw.loc[(bmw['car'] == 'MINI'), 'car_class'] = 'B'
bmw.loc[(bmw['car'] == 'FIAT'), 'car_class'] = 'B'
bmw.loc[(bmw['car'] == 'LAND ROVER'), 'car_class'] = 'M SUV'
bmw.loc[(bmw['car'] == 'BYD'), 'car_class'] = 'C'
bmw.loc[(bmw['car'] == 'BENTLEY'), 'car_class'] = 'S'
bmw.loc[(bmw['car'] == 'JAGUAR'), 'car_class'] = 'S'
bmw.loc[(bmw['car'] == 'ROVER'), 'car_class'] = 'C'
bmw.loc[(bmw['model'] == 'RUS'), 'car_class'] = 'RUS'
bmw.loc[(bmw['car'] == 'VAZ'), 'car_class'] = 'VAZ'
bmw.loc[(bmw['car'] == 'DAEWOO'), 'car_class'] = 'C'

# Drop rows with missing values in the 'car_class' column
bmw.dropna(subset=['car_class'], inplace=True)

# Make a copy of the modified DataFrame
df_fin = bmw.copy()

# Drop the 'plate_y' column from the DataFrame
df_fin = df_fin.drop('plate_y', axis=1)

# Create a new column 'unique_id' by concatenating 'user id' and 'plate_x' columns as strings
df_fin['unique_id'] = df_fin['user id'].astype(str) + '_' + df_fin['plate_x'].astype(str)

# Load the second dataset
acc = pd.read_excel('/Users/dawidarakelyan/Desktop/Capstone final/baza_final.xlsx', sheet_name='պատահարներ')
acc = pd.DataFrame(acc)

# Copy required columns from acc dataframe
clean_acc = acc[['Պատահարի կոդ', 'Տուժած/Ապահովագրված օբյեկտի կոդ', 'Օբյեկտի տեսակ', 'id', 'Տեղի ունենալու ամսաթիվ', 'Հատուցման գումար (համարժեք)']].copy()

# Rename columns to English names
new_column_names = {
    'Պատահարի կոդ': 'Acc_id',
    'Տուժած/Ապահովագրված օբյեկտի կոդ': 'Dam_id',
    'Օբյեկտի տեսակ': 'Type',
    'Տեղի ունենալու ամսաթիվ': 'Date',
    'Հատուցման գումար (համարժեք)': 'Payoff_amount'
}

clean_acc = clean_acc.rename(columns=new_column_names)

import datetime
import pandas as pd

# Get the number of accidents per contract
contract_accident_count = clean_acc.groupby('id')["Acc_id"].nunique().reset_index()
contract_accident_count.columns = ["id", "Total_acc"]

# Get the total number of damaged items per contract
contract_accident_count_item = clean_acc.groupby('id')["Dam_id"].nunique().reset_index()
contract_accident_count_item.columns = ["id", "Total_dam_items"]

# Map the "Type" column to a more readable "Damaged Type" column
clean_acc["Damaged Type"] = clean_acc["Type"].map({"Մեքենա": "car", "Անձ": "human", "Գույք": "object"})

# Get the total amount paid out per type of damage and number of unique damages per contract
contract_damage_counts = clean_acc.groupby(["id", "Damaged Type"]).agg({"Payoff_amount": "sum", "Dam_id": "nunique"})
contract_damage_counts = contract_damage_counts.reset_index()

# Pivot the table to have each type of damage as its own column
contract_damage_counts = contract_damage_counts.pivot(index="id", columns="Damaged Type")
contract_damage_counts.columns = ["_".join(col).rstrip("_") for col in contract_damage_counts.columns.values]
contract_damage_counts = contract_damage_counts.reset_index()

# Replace NaN values with 0 in the table
contract_damage_counts.fillna(0, inplace=True)

# Calculate the total payoff for each contract
contract_damage_counts["Total_payoff"] = contract_damage_counts["Payoff_amount_car"] + contract_damage_counts["Payoff_amount_human"] + contract_damage_counts["Payoff_amount_object"]

# Calculate the minimum, average, and maximum payoffs for each contract
contract_damage_counts["Min_payoff"] = clean_acc.groupby("id")["Payoff_amount"].min().values
contract_damage_counts["Avg_payoff"] = clean_acc.groupby("id")["Payoff_amount"].mean().values
contract_damage_counts["Max_payoff"] = clean_acc.groupby("id")["Payoff_amount"].max().values

# Get the earliest and latest dates of accidents for each contract, and calculate the number of days since each accident
contract_dates = clean_acc.groupby('id')['Date'].agg(['min', 'max']).reset_index()
current_date = datetime.date.today()
contract_dates['First_accident_days'] = (current_date - pd.to_datetime(contract_dates['min']).dt.date).dt.days
contract_dates['Last_accident_days'] = (current_date - pd.to_datetime(contract_dates['max']).dt.date).dt.days

# Merge all of the above tables into one table
user_acc_metrics = contract_accident_count.merge(contract_accident_count_item, on='id').merge(contract_damage_counts, on='id').merge(contract_dates, on='id')

# Merge contract df with user metrics
merged_df = pd.merge(df_fin, user_acc_metrics, on='id', how='left')

# Save the ready file 
merged_df.to_excel('/Users/dawidarakelyan/Desktop/Capstone final/before_metrics.xlsx')

/var/folders/jv/s39xd5dj6fx92jhzp5b_lp8h0000gn/T/ipykernel_20285/2164828780.py:9: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  import pandas_profiling
/var/folders/jv/s39xd5dj6fx92jhzp5b_lp8h0000gn/T/ipykernel_20285/2164828780.py:900: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['model'] = df_filtered.groupby(['car', 'hp'])['model'].fillna(method='ffill')
